In [640]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

In [641]:
sampled_pwds_50 = pd.read_csv("tl/e24_random_50_pwmds.csv")
sampled_pwds_50 = list(sampled_pwds_50['powermode'].values)


sampled_pwds_250 = pd.read_csv("tl/e24_random_250_pwmds.csv")
sampled_pwds_250 = list(sampled_pwds_250['powermode'].values)

In [642]:
# model_train = 'yolo'
# model_infer = 'resnet'
 
# model_train = 'resnet'
# model_infer = 'mobnet'
 
# model_train = 'mobnet'
# model_infer = 'lstm'
 
# model_train = 'mobnet'
# model_infer = 'mobnet'
 
model_train = 'resnet'
model_infer = 'bert'


arr_rate_list = [30, 40, 50, 60, 70, 80, 90, 100, 110, 120]
time_list = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0]
power_list = np.arange(10, 51, 1)

if model_infer == 'bert' or model_train == 'bert':

    time_list = [2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6]
    arr_rate_list = [1,2,3,4,5,6,7,8,9,10, 11, 12, 13, 14, 15]
    power_list = np.arange(10, 61, 1)
    print(len(arr_rate_list)*len(time_list)*len(power_list))

6885


In [643]:
data_train = pd.read_csv(f'{model_train}_train_data_final.csv')
data_infer = pd.read_csv(f'{model_infer}_infer_data_final.csv')

In [644]:
core_vals=[4, 8, 12] #3 possible values
gpu_vals=[114750000, 318750000, 522750000, 726750000, 930750000, 1134750000, 1300500000]
cpu_vals=[422400, 729600, 1036800, 1344000, 1651200, 1958400, 2201600] #in kHz, 7 possible values
mem_vals = [665600000, 2133000000, 3199000000]
bs_vals = [1, 4, 16, 32, 64]

In [645]:
# # Filter data based on values in the list only
data_train = data_train[data_train['cores'].isin(core_vals)]
data_train = data_train[data_train['gpu'].isin(gpu_vals)]
data_train = data_train[data_train['cpu'].isin(cpu_vals)]
data_train = data_train[data_train['mem'].isin(mem_vals)]
data_train.reset_index(drop=True, inplace=True)

In [646]:
def helper(data, arr_rate):
    data['interleaved_power'] = np.maximum(data['observed_power_train'], data['observed_power_infer'])
    data['interleaved_time'] = data['observed_time_train'] + data['observed_time_infer']
    data['powermode'] = data['cores'].astype(str) + "_" + data['cpu'].astype(str) + "_" + data['gpu'].astype(str) + "_" + data['mem'].astype(str)
    data['observed_time_infer_scaled'] = data['observed_time_infer']/1000
    data['observed_time_train_scaled'] = data['observed_time_train']/1000
    data['num_train_batches'] = np.floor(((data['bs'].astype(int))/arr_rate - data['observed_time_infer_scaled'])/data['observed_time_train_scaled']).astype(int)
    data['time_cond1'] = (data['bs'].astype(int))/arr_rate + data['observed_time_infer_scaled']
    data['time_cond2'] = data['observed_time_infer_scaled']*arr_rate

    return data

In [647]:
def get_best_row_optim(data, time_budget, power_budget):

    data = data[data['interleaved_power'] <= power_budget]
    data = data[data['time_cond1'] <= time_budget]
    data = data[data['time_cond2'] <= data['bs']]


    if data.empty:
        return None, None, None, None, None, None
    
    else:     
        data = data.sort_values(by=['num_train_batches'],ascending=False)

        train_batch_vals = data['num_train_batches'].unique()
        max_train_batch_val = max(train_batch_vals)

        data = data[data['num_train_batches'] == max_train_batch_val]
        data.sort_values(by=['observed_time_infer_scaled'], inplace=True)

        data = data.iloc[0]
        powermode = data['powermode']
        bs = data['bs']
        power = data['interleaved_power']
        infer_time = data['observed_time_infer']
        train_time = data['observed_time_train']
        num_train_batches = data['num_train_batches']

        return powermode, bs, power, infer_time, train_time, num_train_batches

In [648]:
def get_best_row_optim_nn(data, time_budget, power_budget):

    data = data[data['predicted_interleaved_power'] <= power_budget]
    data = data[data['predicted_time_cond1'] <= time_budget]
    data = data[data['predicted_time_cond2'] <= data['bs']]


    if data.empty:
        return None, None, None, None, None, None
    
    else:     
        data = data.sort_values(by=['predicted_num_train_batches'],ascending=False)

        train_batch_vals = data['predicted_num_train_batches'].unique()
        max_train_batch_val = max(train_batch_vals)

        data = data[data['predicted_num_train_batches'] == max_train_batch_val]
        data.sort_values(by=['predicted_time_infer_scaled'], inplace=True)

        data = data.iloc[0]
        powermode = data['powermode']
        bs = data['bs']
        power = data['observed_interleaved_power']
        infer_time = data['observed_time_infer']
        train_time = data['observed_time_train']
        num_train_batches = data['observed_num_train_batches']

        return powermode, bs, power, infer_time, train_time, num_train_batches

In [649]:
nn_train_pareto_df = pd.read_csv(f"/home/saisamarth/exp/tl/e24_merged_observed_predicted_{model_train}_NN_sampled_250.csv")
nn_infer_pareto_df = pd.read_csv(f"/home/saisamarth/exp/tl/e24_merged_observed_predicted_{model_train}_NN_infer_sampled_250.csv")
nn_train_pareto_df = nn_train_pareto_df[nn_train_pareto_df['powermode'].isin(sampled_pwds_250)]
nn_infer_pareto_df = nn_infer_pareto_df[nn_infer_pareto_df['powermode'].isin(sampled_pwds_50)]
nn_train_pareto_df.reset_index(inplace=True, drop=True)
nn_infer_pareto_df.reset_index(inplace=True, drop=True)

In [650]:
# merge nn_train_pareto_df and nn_infer_pareto_df based on powermode with suffixes _train and _infer


In [651]:
def helper_nn(data, arr_rate):
    data['observed_interleaved_power'] = np.maximum(data['observed_power_train'], data['observed_power_infer'])
    data['observed_interleaved_time'] = data['observed_time_train'] + data['observed_time_infer']
    data['predicted_interleaved_power'] = np.maximum(data['predicted_power_train'], data['predicted_power_infer'])
    data['predicted_interleaved_time'] = data['predicted_time_train'] + data['predicted_time_infer']

    data['predicted_time_infer_scaled'] = data['predicted_time_infer']/1000
    data['predicted_time_train_scaled'] = data['predicted_time_train']/1000
    data['observed_time_infer_scaled'] = data['observed_time_infer']/1000
    data['observed_time_train_scaled'] = data['observed_time_train']/1000

    data['predicted_num_train_batches'] = np.floor(((data['bs'].astype(int))/arr_rate - data['predicted_time_infer_scaled'])/data['predicted_time_train_scaled']).astype(int)
    data['observed_num_train_batches'] = np.floor(((data['bs'].astype(int))/arr_rate - data['observed_time_infer_scaled'])/data['observed_time_train_scaled']).astype(int)

    data['predicted_time_cond1'] = (data['bs'].astype(int))/arr_rate + data['predicted_time_infer_scaled']
    data['predicted_time_cond2'] = data['predicted_time_infer_scaled']*arr_rate

    data['observed_time_cond1'] = (data['bs'].astype(int))/arr_rate + data['observed_time_infer_scaled']
    data['observed_time_cond2'] = data['observed_time_infer_scaled']*arr_rate

    return data

In [652]:
nn_result_df = pd.DataFrame(columns=['powermode','infer_time','bs','interleaved_power','power_budget','num_train_batches','time_budget', 'arr_rate'])
# time_list = [1]
# power_list = [20]
for arr_rate in arr_rate_list:
    for time_budget in time_list:
        for power_budget in power_list:
            nn_pareto_df = pd.merge(nn_train_pareto_df, nn_infer_pareto_df, on=['powermode'], suffixes=('_train', '_infer'))
            nn_pareto_df = helper_nn(nn_pareto_df, arr_rate)
            pwd, bs, interleaved_power, infer_time, train_time, best_num_train_batches = get_best_row_optim_nn(nn_pareto_df, time_budget, power_budget)
            nn_result_df.loc[len(nn_result_df)] = [pwd, infer_time, bs, interleaved_power,power_budget, best_num_train_batches, time_budget, arr_rate]

In [653]:
random_result_df = pd.DataFrame(columns=['powermode','infer_time','bs','interleaved_power','power_budget','num_train_batches','time_budget','arr_rate'])
# time_list = [1]
# power_list = [20]
for arr_rate in arr_rate_list:
    for time_budget in time_list:
        for power_budget in power_list:
            # merge the data_train and data_infer based on cores, gpu, cpu, mem with suffixes _train and _infer
            data_all = pd.merge(data_train, data_infer, on=['cores', 'gpu', 'cpu', 'mem'], suffixes=('_train', '_infer'))
            data_all = helper(data_all,arr_rate)
            data_all = data_all[data_all['powermode'].isin(sampled_pwds_50)]
            pwd, bs, interleaved_power, infer_time, train_time, best_num_train_batches = get_best_row_optim(data_all, time_budget, power_budget)
            random_result_df.loc[len(random_result_df)] = [pwd, infer_time, bs, interleaved_power,power_budget, best_num_train_batches, time_budget, arr_rate]

In [654]:
# random_result_df.dropna(inplace=True)
# nn_result_df.dropna(inplace=True)
# # random_result_df

In [655]:
# nn_result_df.dropna(inplace=True)

In [656]:
nn_result_df

,powermode,infer_time,bs,interleaved_power,power_budget,num_train_batches,time_budget,arr_rate
0,NaN,NaN,NaN,NaN,10.0,NaN,2.0,1.0
1,NaN,NaN,NaN,NaN,11.0,NaN,2.0,1.0
2,NaN,NaN,NaN,NaN,12.0,NaN,2.0,1.0
3,NaN,NaN,NaN,NaN,13.0,NaN,2.0,1.0
4,NaN,NaN,NaN,NaN,14.0,NaN,2.0,1.0
...,...,...,...,...,...,...,...,...
6880,8_1958400_930750000_3199000000,221.97168,64.0,41.218,56.0,57.0,6.0,15.0
6881,8_1958400_930750000_3199000000,221.97168,64.0,41.218,57.0,57.0,6.0,15.0
6882,8_1958400_930750000_3199000000,221.97168,64.0,41.218,58.0,57.0,6.0,15.0
6883,8_1958400_930750000_3199000000,221.97168,64.0,41.218,59.0,57.0,6.0,15.0


In [657]:
# merge nn_result_df and random_result_df based on power_budget and time_budget with suffixes _nn and _random
result_df = pd.merge(nn_result_df, random_result_df, on=['power_budget', 'time_budget','arr_rate'], suffixes=('_nn', '_random'))

In [658]:
result_df

,powermode_nn,infer_time_nn,bs_nn,interleaved_power_nn,power_budget,num_train_batches_nn,time_budget,arr_rate,powermode_random,infer_time_random,bs_random,interleaved_power_random,num_train_batches_random
0,NaN,NaN,NaN,NaN,10.0,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,11.0,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,12.0,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,13.0,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,14.0,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6880,8_1958400_930750000_3199000000,221.97168,64.0,41.218,56.0,57.0,6.0,15.0,NaN,NaN,NaN,NaN,NaN
6881,8_1958400_930750000_3199000000,221.97168,64.0,41.218,57.0,57.0,6.0,15.0,NaN,NaN,NaN,NaN,NaN
6882,8_1958400_930750000_3199000000,221.97168,64.0,41.218,58.0,57.0,6.0,15.0,4_422400_1300500000_3199000000,1046.325745,16.0,57.1,0.0
6883,8_1958400_930750000_3199000000,221.97168,64.0,41.218,59.0,57.0,6.0,15.0,4_422400_1300500000_3199000000,1046.325745,16.0,57.1,0.0


In [659]:
def validate_nn_solution(data):
    data['flag'] = True
    # check if optim_power_nn is less than power_budget, flag = False
    data.loc[data['interleaved_power_nn'] > (data['power_budget'] + 1), 'flag'] = False
    data['time_scaled'] = data['infer_time_nn']/1000.0
    data['time_cond1'] = (data['bs_nn'])/data['arr_rate'] + data['time_scaled']
    data['time_cond2'] = data['time_scaled']*data['arr_rate']
    # check if time_cond1 is less than time_budget, flag = False
    data.loc[data['time_cond1'] > (data['time_budget'] + 0.01), 'flag'] = False
    # check if time_cond2 is less than bs, flag = False
    data.loc[data['time_cond2'] > data['bs_nn'], 'flag'] = False
    return data


In [663]:
result_df = validate_nn_solution(result_df)

In [665]:
if model_infer == 'bert' or model_train == 'bert':
    result_df.dropna(inplace=True)

In [668]:
result_df.to_csv(f'interleaved_{model_train}_train_{model_infer}_infer_nn_random_multi_al.csv', index=False)